In [1]:
#import packages
from sympy import *
from sympy.physics.quantum import TensorProduct
init_session()

IPython console for SymPy 1.8 (Python 3.8.8-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.8/



In [2]:
#constructing generator matrix
q = symbols('q')
n = symbols('n')
L1 = Matrix([[-1*(q**(2*n-2)-q**(2*n-4)+(2/q**2)+((-q**(1-n)+q**(n-1))**2/q**2)+q**(2*n-2)-q**(2*n-4)+(2/q**2)),
              q**(2*n-2)-q**(2*n-4)+(2/q**2),(-q**(1-n)+q**(n-1))**2/q**2,q**(2*n-2)-q**(2*n-4)+(2/q**2)],
             
             [q**(2*n)-q**(2*n-2)+2,
              -1*(q**(2*n)-q**(2*n-2)+2+q**(-2*n)-q**(2-2*n)+2+(-q**(1-n)+q**(n-1))**2),
              q**(-2*n)-q**(2-2*n)+2,(-q**(1-n)+q**(n-1))**2],
             
            [(q**2)*(-q**(1-n)+q**(n-1))**2,2*q**2+q**(2-2*n)-q**(4-2*n),
             -1*(((q**2)*(-q**(1-n)+q**(n-1))**2)+2*q**2+q**(2-2*n)-q**(4-2*n)+2*q**2+q**(2-2*n)-q**(4-2*n)),
             2*q**2+q**(2-2*n)-q**(4-2*n)],
             
            [q**(2*n)-q**(2*n-2)+2,(-q**(1-n)+q**(n-1))**2,q**(-2*n)-q**(2-2*n)+2,
             -1*(q**(2*n)-q**(2*n-2)+2+(-q**(1-n)+q**(n-1))**2+q**(-2*n)-q**(2-2*n)+2)]
            
            ])
#4x4 piece given in 2023 REU Paper

L2 = Matrix([[-1*((q**(1-2*n))+(q**(2*n-1)))/q,((q**(1-2*n))+(q**(2*n-1)))/q],
             [q*(q**(1-2*n)+q**(2*n-1)),-q*(q**(1-2*n)+q**(2*n-1))]])
#2x2 piece given in 2023 REU Paper

L2Piece = Matrix.diag(L2,L2)
fullL2 = Matrix.diag(L2Piece,L2Piece)
#Matrix.diag is a way of taking the Direct Product of two matrices

zeroMatrixPiece = Matrix.diag(Matrix([[0]]),Matrix([[0]]))
fullZeroMatrix = Matrix.diag(zeroMatrixPiece,zeroMatrixPiece)

firstPiece = Matrix.diag(L1,fullL2)

generatorMatrix = Matrix.diag(firstPiece,fullZeroMatrix)

In [3]:
#Lambda and Phi Matrix
lambdaMatrix = Matrix([
    [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,q**2/(q**2+1),1/(q**2+1),0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,q**2/(q**2+1),1/(q**2+1),0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
    [q**4/(1+q**2)**2,q**2/(1+q**2)**2,1/(1+q**2)**2,q**2/(1+q**2)**2,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,q**2/(q**2+1),1/(q**2+1),0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,q**2/(q**2+1),1/(q**2+1),0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
])

phiMatrix = Matrix([
    [0,0,0,0,1,0,0,0,0],
    [0,0,0,0,1,0,0,0,0],
    [0,0,0,0,1,0,0,0,0],
    [0,0,0,0,1,0,0,0,0],
    [0,1,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1,0],
    [1,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0],
    [0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,0,0,1],
])

#Tensoring the two matrices with themselves
tensorLambda = TensorProduct(lambdaMatrix,lambdaMatrix)
tensorPhi = TensorProduct(phiMatrix,phiMatrix)

In [4]:
#Tensoring the 16x16 Identity Matrix with the 16x16 generator matrix created in the first code cell
#Type D ASEP generator for 4 lattice sites where only the middle two sites interact
megaGeneratorMatrix = TensorProduct(eye(16),generatorMatrix)

In [5]:
#Reordering the Type D ASEP generator to have the same ordering as the tensored lambda and phi matrix
#get the reordering to line up the matrices to multiply in the right order

#List of all the states of Type D ASEP on two lattice sites
generatorList = ['30','21','03','12','10','01','20','02','31','13','32','23','00','11','22','33']

#The ordering of states for tensored phi and lambda
tensoredOrdering = string_list = ["String {}".format(i) for i in range(256)]

for i in range(16):
    for j in range(16):
        tensoredOrdering[16*i+j] = generatorList[i] + generatorList[j]
        
#the original ordering of states for Type D ASEP on 4 lattice sites,
#the ordering inherited by tensoring the identity with the generator
megaGeneratorOrdering = string_list = ["String {}".format(i) for i in range(256)]

for i in range(16):
    for j in range(16):
        megaGeneratorOrdering[16*i+j] = generatorList[i][0] + generatorList[j] + generatorList[i][1]
        
#the indexes needed to map the megaGeneratorOrdering => tensoredOrdering
indexOrderingList = [0 for _ in range(256)]
 
for i in range(256):
       indexOrderingList[i] = megaGeneratorOrdering.index(tensoredOrdering[i])

#reordering of the states of the Type D ASEP generator on 4 lattice sites to match with the tensored phi and lambda
megaGeneratorMatrix = megaGeneratorMatrix[indexOrderingList,indexOrderingList]

In [6]:
#Calculating the Stochastic Fusion Matrix for Type D ASEP on 4 lattice sites
fusionMatrix = tensorLambda*megaGeneratorMatrix*tensorPhi
#Simplifying the matrix
simplifyFusionMatrix = fusionMatrix.applyfunc(simplify)

In [7]:
#Diagonalizing the Stochastic Fusion Matrix
import numpy as np
#Ordering of the Fused States
fusionList = ['0','1','2','11','3','22','31','32','33']
megaFusionList = string_list = ["String {}".format(i) for i in range(81)]

for i in range(9):
    for j in range(9):
        megaFusionList[9*i+j] = fusionList[i] + '_' + fusionList[j]
        
#Ordering by communicating class, puts the matrix in block diagonal form      
diagonalOrdering = ['3_3','0_33','33_0','1_32','32_1','2_31','31_2','11_22','22_11',
                    '1_3','3_1','0_31','31_0','2_11','11_2',
                   '2_3','3_2','0_32','32_0','1_22','22_1',
                   '3_31','31_3','1_33','33_1','11_32','32_11',
                   '3_32','32_3','2_33','33_2','22_31','31_22',
                   '3_0','1_2','2_1','0_3',
                   '31_1','11_3','3_11','1_31',
                   '32_2','22_3','3_22','2_32',
                   '33_3','31_32','32_31','3_33',
                   '1_1','0_11','11_0',
                   '2_2','0_22','22_0',
                   '31_31','11_33','33_11',
                   '32_32','22_33','33_22',
                   '0_1','1_0',
                   '0_2','2_0',
                   '1_11','11_1',
                   '2_22','22_2',
                   '11_31','31_11',
                   '22_32','32_22',
                   '31_33','33_31',
                   '32_33','33_32',
                   '0_0',
                   '11_11',
                   '22_22',
                   '33_33']
diagonalIndexList = []
for item in diagonalOrdering:
    diagonalIndexList.append(megaFusionList.index(item))

diagonalFusion = simplifyFusionMatrix[diagonalIndexList,diagonalIndexList]

In [9]:
#printing the block diagonalized Stochastic Fusion Matrix
diagonalFusion

⎡                                                                             
⎢ -2⋅n - 4 ⎛ 12      10      8      6    4    4⋅n      2⋅n + 2      2⋅n + 4   
⎢q        ⋅⎝q   - 4⋅q   + 3⋅q  - 2⋅q  - q  + q    - 2⋅q        + 4⋅q        - 
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                            q
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                         2 ⎛ 
⎢                                                                        q ⋅⎝q
⎢                                                                             
⎢                                                   